# TensorRider Algorithm
Building regression model with TensorFlow's native APIs.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

BATCH_SIZE = 256
EPOCHS = 300
L_RATE = 0.01

# def imgPreprocess(rgb):
#     r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
#     # tgt = 0.2989 * r + 0.5870 * g + 0.1140 * b
#     tgt = 0.5 * r + 0.25 * g + 0.25 * b
#     return tgt

# Prepare

In [2]:
def decode(serialized_example):
    
    features = tf.parse_single_example(
            serialized_example,
            features={
                    'image': tf.FixedLenFeature([], tf.string),
                    'label': tf.FixedLenFeature([], tf.int64),
            })

    image = tf.decode_raw(features['image'], tf.uint8)
    image.set_shape((4800))

    label = tf.cast(features['label'], tf.int32)
    
    return image, label


def normalize(image, label):
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
    return image, label

def inputs(file, batch_size, num_epochs):

    filename = file + '.tfrecords'

    with tf.name_scope('input'):
        
        dataset = tf.data.TFRecordDataset(filename)

        dataset = dataset.map(decode)
        dataset = dataset.map(normalize)

        dataset = dataset.shuffle(1000 + 3 * batch_size)

        dataset = dataset.repeat(num_epochs)
        dataset = dataset.batch(batch_size)

        iterator = dataset.make_one_shot_iterator()
        x, y = iterator.get_next()
        x = tf.reshape(x, [BATCH_SIZE, 14400])
        y = tf.reshape(y, [BATCH_SIZE, 1])
        
    return x, y

sess = tf.InteractiveSession()
x, y = inputs('train', BATCH_SIZE, EPOCHS)
print(tf.shape(x))
print(tf.shape(y))
print(tf.Tensor.get_shape(x))
print(tf.Tensor.get_shape(y))

ximg = np.array(sess.run(x))
print(ximg.shape)
ximg = ximg.reshape((BATCH_SIZE, 60, 80, 3))
plt.imshow(ximg[0,:,:,:])

# Define the Dense Layers

In [3]:
# tf Graph input
X = tf.placeholder("float", [None, 14400])
Y = tf.placeholder("float", [None, 1])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([14400, 2048])),
    'h2': tf.Variable(tf.random_normal([2048, 1024])),
    'out': tf.Variable(tf.random_normal([1024, 1]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([2048])),
    'b2': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([1]))
}

# Create model
def neural_net(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = neural_net(X)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
# loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#     logits=logits, labels=Y))
loss_op = tf.reduce_mean(tf.losses.mean_squared_error(Y, prediction))
optimizer = tf.train.AdamOptimizer(learning_rate=L_RATE)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.greater(0.250, prediction - Y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Train

In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, EPOCHS + 1):
        batch_x, batch_y = sess.run(inputs('train', BATCH_SIZE, EPOCHS))
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % 5 == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))

Step 1, Minibatch Loss= 128.2500, Training Accuracy= 0.285
Step 5, Minibatch Loss= 112.1328, Training Accuracy= 0.297
Step 10, Minibatch Loss= 112.1523, Training Accuracy= 0.328
Step 15, Minibatch Loss= 118.3281, Training Accuracy= 0.309
Step 20, Minibatch Loss= 121.1914, Training Accuracy= 0.305
Step 25, Minibatch Loss= 125.1016, Training Accuracy= 0.289
Step 30, Minibatch Loss= 141.1406, Training Accuracy= 0.297
Step 35, Minibatch Loss= 118.2969, Training Accuracy= 0.324
Step 40, Minibatch Loss= 107.1250, Training Accuracy= 0.320
Step 45, Minibatch Loss= 130.0742, Training Accuracy= 0.285
Step 50, Minibatch Loss= 124.0742, Training Accuracy= 0.301
Step 55, Minibatch Loss= 120.6094, Training Accuracy= 0.305
Step 60, Minibatch Loss= 113.6523, Training Accuracy= 0.285
Step 65, Minibatch Loss= 129.1797, Training Accuracy= 0.328
Step 70, Minibatch Loss= 122.9219, Training Accuracy= 0.273
Step 75, Minibatch Loss= 123.4336, Training Accuracy= 0.336
Step 80, Minibatch Loss= 129.2617, Trainin